In [22]:
import tensorflow as tf
from keras import models, datasets
import numpy as np

In [23]:
(x_train_full, y_train_full), (x_test, y_test) = datasets.cifar10.load_data()

In [24]:
model = models.load_model("best_model.keras")

In [25]:
def clone_model_with_weights(model):
    clone = tf.keras.models.clone_model(model)
    clone.set_weights(model.get_weights())

    clone.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return clone


def mutate_weights_gaussian(model, sigma=0.002, fraction=0.02):
    mutated = clone_model_with_weights(model)
    new_weights = []
    for w in mutated.get_weights():
        mask = (np.random.rand(*w.shape) < fraction)
        noise = np.random.normal(0, sigma, size=w.shape).astype(w.dtype)
        new_weights.append(w + noise * mask)
    mutated.set_weights(new_weights)
    return mutated


def mutate_weight_sign_flip(model, fraction=0.01):
    mutated = clone_model_with_weights(model)
    new_weights = []
    for w in mutated.get_weights():
        w2 = w.copy()
        mask = (np.random.rand(*w.shape) < fraction)
        w2[mask] *= -1
        new_weights.append(w2)
    mutated.set_weights(new_weights)
    return mutated


def mutate_weight_scaling(model, fraction=0.02, scale=0.15):
    mutated = clone_model_with_weights(model)
    new_weights = []
    for w in mutated.get_weights():
        w2 = w.copy()
        mask = (np.random.rand(*w.shape) < fraction)
        perturbation = np.random.uniform(-scale, scale, size=w.shape).astype(w.dtype)
        w2[mask] *= 1.0 + perturbation[mask]
        new_weights.append(w2)
    mutated.set_weights(new_weights)
    return mutated


def mutate_bias_noise(model, sigma=0.01, fraction=0.3):
    mutated = clone_model_with_weights(model)
    new_weights = []
    for w in mutated.get_weights():
        if w.ndim != 1:
            new_weights.append(w)
            continue
        w2 = w.copy()
        mask = (np.random.rand(*w.shape) < fraction)
        noise = np.random.normal(0, sigma, size=w.shape).astype(w.dtype)
        w2[mask] += noise[mask]
        new_weights.append(w2)
    mutated.set_weights(new_weights)
    return mutated


def eval_model_mutation(model, x, y, mut_fn, name, baseline_acc, **kwargs):
    mutated_model = mut_fn(model, **kwargs)
    loss, acc = mutated_model.evaluate(x, y, verbose=0)
    return {"mutation": name, "acc": float(acc), "acc_drop": float(baseline_acc - acc)}


In [26]:
baseline_loss, baseline_acc = model.evaluate(x_test, y_test, verbose=0)

results = []
results.append({"mutation": "baseline", "acc": float(baseline_acc), "acc_drop": 0.0})

results.append(eval_model_mutation(model, x_test, y_test, mutate_weights_gaussian,
                                   "weight_noise_sigma_0.002", baseline_acc))

results.append(eval_model_mutation(model, x_test, y_test, mutate_weight_sign_flip,
                                   "sign_flip_1pct", baseline_acc))

results.append(eval_model_mutation(model, x_test, y_test, mutate_weight_scaling,
                                   "weight_scaling_2pct", baseline_acc))

results.append(eval_model_mutation(model, x_test, y_test, mutate_bias_noise,
                                   "bias_noise_30pct", baseline_acc))

results


[{'mutation': 'baseline', 'acc': 0.9099000096321106, 'acc_drop': 0.0},
 {'mutation': 'weight_noise_sigma_0.002',
  'acc': 0.9050999879837036,
  'acc_drop': 0.004800021648406982},
 {'mutation': 'sign_flip_1pct',
  'acc': 0.10729999840259552,
  'acc_drop': 0.8026000112295151},
 {'mutation': 'weight_scaling_2pct',
  'acc': 0.9104999899864197,
  'acc_drop': -0.000599980354309082},
 {'mutation': 'bias_noise_30pct',
  'acc': 0.7685999870300293,
  'acc_drop': 0.1413000226020813}]